# Feature engineering

1. Работа с выбросами
2. Отсутствующие (пропущенные) значения
3. Категориальные признаки

## Как работать с выбросами (outliers)

В статистике "выброс" (outlier) - это точка данных, которая существенно отличается от других точек.

Выбросы могут возникать из-за ошибок измерения, или например из-за ошибок эксперимента (в этом случае их иногда исключают из набора данных).

Выбросы могут приводит к проблемам в применении статистических методов анализа данных.

Даже если точка данных является выбросом, это всё ещё точка данных!  Аккуратно изучайте данные, откуда они пришли, и как Вы планируете использовать эти данные - и только тогда решайте, удалять или не удалять те или иные выбросы. Случаи бывают разные!



### Загрузка библиотек

In [13]:
import numpy as np # математические вычисления
import pandas as pd # работа с таблицами
import matplotlib
import matplotlib.pyplot as plt # визуализация данных
import plotly.express as px # визуализация данных

### Генерируем данные

In [14]:
def create_ages(mu=50,sigma=13,num_samples=100,seed=42):
    # Генерация вызовов для числовых данных
    # Указываем среднее значение, среднеквадратическое отклонение и количество сэмплов
    np.random.seed(seed) # Для получения тех же самых данных

    sample_ages = np.random.normal(loc=mu,scale=sigma,size=num_samples)
    sample_ages = np.round(sample_ages,decimals=0)

    return sample_ages

In [15]:
sample = pd.DataFrame({'age': create_ages()}) # Генерация вызовов для числовых данных
sample

,age
0,56.0
1,48.0
2,58.0
3,70.0
4,47.0
...,...
95,31.0
96,54.0
97,53.0
98,50.0


### Визуализация и описание данных

In [16]:
px.histogram(sample,nbins=10)

In [17]:
px.box(sample)

In [18]:
sample.describe().T

,count,mean,std,min,25%,50%,75%,max
age,100.0,48.66,11.82039,16.0,42.0,48.0,55.25,74.0


### Усечение или исправление данных на основе знаний о данных

Если мы знаем, что работаем с данными только тех, кто имеет право голосовать (с 18 лет), то мы можем удалить все значения меньше 18 лет, ИЛИ заменить такие значения значением 18.

In [19]:
sample = sample[sample > 18]

In [20]:
# Удалить одного человека
sample.describe()

,age
count,99.000000
mean,48.989899
std,11.408460
min,24.000000
25%,42.500000
50%,48.000000
75%,55.500000
max,74.000000


In [21]:
def fix_values(df):
    # Функция для исправления возраста
    df.loc[df['age'] < 18, 'age'] = 18
    return df

In [22]:
# Применяем функцию к DataFrame
sample_fixed = fix_values(sample)
sample_fixed.describe()

,age
count,99.000000
mean,48.989899
std,11.408460
min,24.000000
25%,42.500000
50%,48.000000
75%,55.500000
max,74.000000


In [23]:
len(sample_fixed)

100

--------

Есть много способов найти и удалить выбросы:
* Усечение данных на основе некоторого порогового значения
* Усечение на основе IQR (interquantile range - межквантильный размах) или STD (standard deviation - среднеквадратическое отклонение)

### Набор данных "Ames Data Set"

Давайте найдём выбросы в нашем наборе данных Ames Housing Data Set

https://www.kaggle.com/datasets/prevek18/ames-housing-dataset

In [24]:
df = pd.read_csv("/content/AmesHousing.csv")

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2930 entries, 0 to 2929
Data columns (total 82 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Order            2930 non-null   int64  
 1   PID              2930 non-null   int64  
 2   MS SubClass      2930 non-null   int64  
 3   MS Zoning        2930 non-null   object 
 4   Lot Frontage     2440 non-null   float64
 5   Lot Area         2930 non-null   int64  
 6   Street           2930 non-null   object 
 7   Alley            198 non-null    object 
 8   Lot Shape        2930 non-null   object 
 9   Land Contour     2930 non-null   object 
 10  Utilities        2930 non-null   object 
 11  Lot Config       2930 non-null   object 
 12  Land Slope       2930 non-null   object 
 13  Neighborhood     2930 non-null   object 
 14  Condition 1      2930 non-null   object 
 15  Condition 2      2930 non-null   object 
 16  Bldg Type        2930 non-null   object 
 17  House Style   

In [26]:
# Выбираем только числовые столбцы
numerical_df = df.select_dtypes(include=['int64', 'float64'])

# Вычисляем корреляцию
corr_matrix = numerical_df.corr()

# Создаем тепловую карту
fig = px.imshow(
    corr_matrix,
    text_auto=True,
    width=1200,  # Увеличиваем ширину до 800 пикселей
                height=1200  # Увеличиваем высоту до 600 пикселей
    )
fig.show()

In [27]:
corr_matrix['SalePrice'].sort_values() # SalePrice - стоимость

,SalePrice
PID,-0.246521
Enclosed Porch,-0.128787
Kitchen AbvGr,-0.119814
Overall Cond,-0.101697
MS SubClass,-0.085092
Low Qual Fin SF,-0.037660
Bsmt Half Bath,-0.035835
Order,-0.031408
Yr Sold,-0.030569
Misc Val,-0.015691


In [28]:
px.histogram(df["SalePrice"],nbins=10)

In [29]:
px.scatter(x='Overall Qual',y='SalePrice',data_frame=df)

In [30]:
df[(df['Overall Qual']>8) & (df['SalePrice']<200000)] # редко встречающиеся значения, возможно выбросы

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
1182,1183,533350090,60,RL,NaN,24572,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,6,2008,WD,Family,150000
1498,1499,908154235,60,RL,313.0,63887,Pave,NaN,IR3,Bnk,...,480,Gd,NaN,NaN,0,1,2008,New,Partial,160000
2180,2181,908154195,20,RL,128.0,39290,Pave,NaN,IR1,Bnk,...,0,NaN,NaN,Elev,17000,10,2007,New,Partial,183850
2181,2182,908154205,60,RL,130.0,40094,Pave,NaN,IR1,Bnk,...,0,NaN,NaN,NaN,0,10,2007,New,Partial,184750


In [31]:
px.scatter(x='Gr Liv Area',y='SalePrice',data_frame=df) #зона проживания

In [32]:
df[(df['Gr Liv Area']>4000) & (df['SalePrice']<400000)]  # редко встречающиеся значения, возможно выбросы

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
1498,1499,908154235,60,RL,313.0,63887,Pave,NaN,IR3,Bnk,...,480,Gd,NaN,NaN,0,1,2008,New,Partial,160000
2180,2181,908154195,20,RL,128.0,39290,Pave,NaN,IR1,Bnk,...,0,NaN,NaN,Elev,17000,10,2007,New,Partial,183850
2181,2182,908154205,60,RL,130.0,40094,Pave,NaN,IR1,Bnk,...,0,NaN,NaN,NaN,0,10,2007,New,Partial,184750


In [33]:
df[(df['Gr Liv Area']>4000) & (df['SalePrice']<400000)].index

Index([1498, 2180, 2181], dtype='int64')

In [34]:
ind_drop = df[(df['Gr Liv Area']>4000) & (df['SalePrice']<400000)].index

In [35]:
df = df.drop(ind_drop,axis=0)

In [36]:
px.scatter(x='Gr Liv Area',y='SalePrice',data_frame=df)

In [37]:
px.scatter(x='Overall Qual',y='SalePrice',data_frame=df)

In [38]:
df.to_csv("Ames_outliers_removed.csv",index=False)

## Работа с отсутствующими данными (пропущенными данными, Missing Data)



Ранее мы уже видели операции в Pandas для отсутствующих данных.
Не существует единственного стопроцентного способа сделать такую очистку правильно.
Приведём один из примеров того, как это можно сделать для конкретного файла с данными.

Замечание №1: Будем постепенно очищать набор данных Ames Housing Dataset и добавлять в него новые признаки, чтобы работать с ними в следующем разделе. Пожалуйста следите в этом блокноте за тем, чтобы загружать данные из файла с одним и тем же названием.
 Замечание №2: Некоторые применённые здесь методы могут не привести к оптимальной производительности модели, они приведены для демонстрации различных доступных методов.

### Данные


In [39]:
df = pd.read_csv("/content/Ames_outliers_removed.csv")

In [40]:
df.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [41]:
len(df.columns)

82

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2927 entries, 0 to 2926
Data columns (total 82 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Order            2927 non-null   int64  
 1   PID              2927 non-null   int64  
 2   MS SubClass      2927 non-null   int64  
 3   MS Zoning        2927 non-null   object 
 4   Lot Frontage     2437 non-null   float64
 5   Lot Area         2927 non-null   int64  
 6   Street           2927 non-null   object 
 7   Alley            198 non-null    object 
 8   Lot Shape        2927 non-null   object 
 9   Land Contour     2927 non-null   object 
 10  Utilities        2927 non-null   object 
 11  Lot Config       2927 non-null   object 
 12  Land Slope       2927 non-null   object 
 13  Neighborhood     2927 non-null   object 
 14  Condition 1      2927 non-null   object 
 15  Condition 2      2927 non-null   object 
 16  Bldg Type        2927 non-null   object 
 17  House Style   

### Удаление колонки PID

У нас уже есть индекс, поэтому для работы регрессии нам не нужен уникальный идентификатор PID.

In [43]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Order,2927.0,1.464999e+03,8.461886e+02,1.0,732.5,1464.0,2198.5,2.930000e+03
PID,2927.0,7.142660e+08,1.887256e+08,526301100.0,528477015.0,535453200.0,907180125.0,1.007100e+09
MS SubClass,2927.0,5.739836e+01,4.265422e+01,20.0,20.0,50.0,70.0,1.900000e+02
Lot Frontage,2437.0,6.907550e+01,2.278727e+01,21.0,58.0,68.0,80.0,3.130000e+02
Lot Area,2927.0,1.010937e+04,7.782876e+03,1300.0,7439.0,9430.0,11523.0,2.152450e+05
Overall Qual,2927.0,6.090878e+00,1.406195e+00,1.0,5.0,6.0,7.0,1.000000e+01
Overall Cond,2927.0,5.563717e+00,1.111960e+00,1.0,5.0,5.0,6.0,9.000000e+00
Year Built,2927.0,1.971319e+03,3.023849e+01,1872.0,1954.0,1973.0,2000.5,2.010000e+03
Year Remod/Add,2927.0,1.984242e+03,2.085673e+01,1950.0,1965.0,1993.0,2004.0,2.010000e+03
Mas Vnr Area,2904.0,1.010441e+02,1.770987e+02,0.0,0.0,0.0,164.0,1.600000e+03


In [44]:
df = df.drop('PID',axis=1)

In [45]:
len(df.columns)

81

### Признаки со значениями NaN

In [46]:
df.isnull()

,Order,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,False,False,False,False,False,False,True,False,False,False,...,False,True,True,True,False,False,False,False,False,False
1,False,False,False,False,False,False,True,False,False,False,...,False,True,False,True,False,False,False,False,False,False
2,False,False,False,False,False,False,True,False,False,False,...,False,True,True,False,False,False,False,False,False,False
3,False,False,False,False,False,False,True,False,False,False,...,False,True,True,True,False,False,False,False,False,False
4,False,False,False,False,False,False,True,False,False,False,...,False,True,False,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2922,False,False,False,False,False,False,True,False,False,False,...,False,True,False,True,False,False,False,False,False,False
2923,False,False,False,True,False,False,True,False,False,False,...,False,True,False,True,False,False,False,False,False,False
2924,False,False,False,False,False,False,True,False,False,False,...,False,True,False,False,False,False,False,False,False,False
2925,False,False,False,False,False,False,True,False,False,False,...,False,True,True,True,False,False,False,False,False,False


In [47]:
df.isnull().sum()

,0
Order,0
MS SubClass,0
MS Zoning,0
Lot Frontage,490
Lot Area,0
...,...
Mo Sold,0
Yr Sold,0
Sale Type,0
Sale Condition,0


In [48]:
100* df.isnull().sum() / len(df)

,0
Order,0.00000
MS SubClass,0.00000
MS Zoning,0.00000
Lot Frontage,16.74069
Lot Area,0.00000
...,...
Mo Sold,0.00000
Yr Sold,0.00000
Sale Type,0.00000
Sale Condition,0.00000


In [49]:
def percent_missing(df):
    percent_nan = 100* df.isnull().sum() / len(df)
    percent_nan = percent_nan[percent_nan>0].sort_values()
    return percent_nan

In [50]:
percent_nan = percent_missing(df)
percent_nan

,0
Electrical,0.034165
Garage Cars,0.034165
BsmtFin SF 1,0.034165
Garage Area,0.034165
BsmtFin SF 2,0.034165
Bsmt Unf SF,0.034165
Total Bsmt SF,0.034165
Bsmt Half Bath,0.068329
Bsmt Full Bath,0.068329
Mas Vnr Area,0.785787


In [51]:
px.bar(x=percent_nan.index,y=percent_nan)

### Удаление признаков или удаление строк

Если значения отсутствуют только в нескольких строках, количество которых мало по сравнению с общим количеством строк, то можно рассмотреть вариант удалить такие строки. К чему это приведёт с точки зрения точности работы модели? По сути, мы удаляем некоторые данные для обучения и тестирования, но поскольку таких строк очень мало, то скорее всего мы не сильно повлияем на точность модели.

Если же значения отсутствуют почти во всех строках, то имеет смысл полностью удалить такие признаки. Однако перед этим следует внимательно разобраться, почему неопределённых значений так много. В некоторых случаях можно рассмотреть такие данные как отдельную категорию, отдельно от остальных данных.

В качестве примера возьмём признак "количество машин, которые могут поместиться в гараже". Но если гаража вообще нет, то вместо значения 0 может проставляться неопределённое значение null. В таким случае имеет смысл заменить значения null на нули. Только Вы можете решить, что делать с неопределёнными значениями! Используйте Ваши знания о предметной области и специфике данных!

### Работа с отсутствующими данными в строках

#### Заменять данные чем-то, или удалять данные?

Давайте поймём, в каких случаях лучше заменять отсутствующие данные какими-то значениями (например, средним значением), а в каких случаях удалять такие строки. Давайте выберем некоторое пороговое значение (threshold) и договоримся, что при его привышении мы решаем удалять строки (вместо того, чтобы заменять отсутствующее значение каким-то другим значением). Мы выберем пороговое значение 1%. Это значит, что если меньше 1% строк содержат неопределённое значение какого-то признака, то мы просто удалим такие строки. Здесь нет правильного ответа на вопрос, какое следует выбрать пороговое значение. Можете использовать знания о специфике предметной области, но в любом случае пороговое значение не должно быть очень большим, например 50%.

Глядя на текстовое описание признаков, мы видим, что в нашем случае значения NaN проставляются намеренно - они означают 0 или "нисколько".

### Пример замены (заполнения) данных

In [52]:
px.bar(x=percent_nan.index,y=percent_nan) # смотрим значения, где y выше 1 процента

Далее на основе этих данных мы или удалим строки, или заменим отсутствующие данные некоторыми значениями.

In [53]:
# Сравниваем с пороговым значением, в процентах
percent_nan[percent_nan < 1]

,0
Electrical,0.034165
Garage Cars,0.034165
BsmtFin SF 1,0.034165
Garage Area,0.034165
BsmtFin SF 2,0.034165
Bsmt Unf SF,0.034165
Total Bsmt SF,0.034165
Bsmt Half Bath,0.068329
Bsmt Full Bath,0.068329
Mas Vnr Area,0.785787


In [54]:
100/len(df)

0.0341646737273659

In [55]:
df[df['Total Bsmt SF'].isnull()]

,Order,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
1341,1342,20,RM,99.0,5940,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,4,2008,ConLD,Abnorml,79000


In [56]:
df[df['Bsmt Half Bath'].isnull()]

,Order,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
1341,1342,20,RM,99.0,5940,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,4,2008,ConLD,Abnorml,79000
1497,1498,20,RL,123.0,47007,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,7,2008,WD,Normal,284700


**Заполняем данные на основе названий колонок. Здесь у нас есть два типа 2 признаков - числовые признаки и текстовые описания.**

Числовые колонки:

In [57]:
bsmt_num_cols = ['BsmtFin SF 1', 'BsmtFin SF 2', 'Bsmt Unf SF','Total Bsmt SF', 'Bsmt Full Bath', 'Bsmt Half Bath']
df[bsmt_num_cols] = df[bsmt_num_cols].fillna(0) # fillna - заполнение значений, 0 для заполнения

Текстовые колонки:

In [58]:
bsmt_str_cols =  ['Bsmt Qual', 'Bsmt Cond', 'Bsmt Exposure', 'BsmtFin Type 1', 'BsmtFin Type 2']
df[bsmt_str_cols] = df[bsmt_str_cols].fillna('None')

In [59]:
percent_nan = percent_missing(df)

In [60]:
px.bar(x=percent_nan.index,y=percent_nan)

### Удаление строк

Некоторые признаки имеют только одну-две строки с отсутствующими значениями. На основе файла .txt с описанием признаков нашего набора данных, мы могли бы легко заполнить эти данные некоторыми значениями, и это было бы отличное решение. Однако здесь давайте покажем Вам, как удалять такие строки - это пригодится в тех случаях, когда у Вас нет хорошей интерпретации того, что означают отсутствующие значения.

    df.dropna() ---
        subset : array-like, optional
                Labels along other axis to consider, e.g. if you are dropping rows
                these would be a list of columns to include.

In [61]:
df = df.dropna(axis=0,subset= ['Electrical','Garage Cars']) # мало пропущенных, удаляем строки
df

,Order,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2922,2926,80,RL,37.0,7937,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,GdPrv,NaN,0,3,2006,WD,Normal,142500
2923,2927,20,RL,NaN,8885,Pave,NaN,IR1,Low,AllPub,...,0,NaN,MnPrv,NaN,0,6,2006,WD,Normal,131000
2924,2928,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal,132000
2925,2929,20,RL,77.0,10010,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2006,WD,Normal,170000


In [62]:
percent_nan = percent_missing(df)
percent_nan

,0
Mas Vnr Area,0.786325
Garage Type,5.367521
Garage Yr Blt,5.401709
Garage Finish,5.401709
Garage Qual,5.401709
Garage Cond,5.401709
Lot Frontage,16.752137
Fireplace Qu,48.547009
Mas Vnr Type,60.615385
Fence,80.478632


In [63]:
px.bar(x=percent_nan.index,y=percent_nan)

### Признак "Mas Vnr"

На основе текстового описания набора данных, отсутствие данных в признаках Mas Vnr Type и Mas Vnr Area скорее всего означает, что дом не имеет облицовки каменной плиткой, и в этом случае мы укажем нулевое значение, как мы делали раньше для других признаков.

In [64]:
df["Mas Vnr Type"] = df["Mas Vnr Type"].fillna("None")
df["Mas Vnr Area"] = df["Mas Vnr Area"].fillna(0)

In [65]:
percent_nan = percent_missing(df)
percent_nan

,0
Garage Type,5.367521
Garage Yr Blt,5.401709
Garage Finish,5.401709
Garage Qual,5.401709
Garage Cond,5.401709
Lot Frontage,16.752137
Fireplace Qu,48.547009
Fence,80.478632
Alley,93.230769
Misc Feature,96.410256


In [66]:
px.bar(x=percent_nan.index,y=percent_nan)

### Работа с отсутствующими данными в колонках

Ранее мы смотрели на отсутствие данных в строках; теперь посмотрим на колонки признаков, поскольку здесь есть достаточно большой процент отсутствующих значений.

#### Колонки Garage

Судя по описанию данных, значение NaN означает отсутствие гаража, так что мы запишем значение "None" или 0.

In [67]:
df[['Garage Type', 'Garage Finish', 'Garage Qual', 'Garage Cond']]

,Garage Type,Garage Finish,Garage Qual,Garage Cond
0,Attchd,Fin,TA,TA
1,Attchd,Unf,TA,TA
2,Attchd,Unf,TA,TA
3,Attchd,Fin,TA,TA
4,Attchd,Fin,TA,TA
...,...,...,...,...
2922,Detchd,Unf,TA,TA
2923,Attchd,Unf,TA,TA
2924,NaN,NaN,NaN,NaN
2925,Attchd,RFn,TA,TA


In [68]:
gar_str_cols = ['Garage Type', 'Garage Finish', 'Garage Qual', 'Garage Cond']
df[gar_str_cols] = df[gar_str_cols].fillna('None')

In [69]:
df['Garage Yr Blt'] = df['Garage Yr Blt'].fillna(0)

In [70]:
percent_nan = percent_missing(df)
percent_nan

,0
Lot Frontage,16.752137
Fireplace Qu,48.547009
Fence,80.478632
Alley,93.230769
Misc Feature,96.410256
Pool QC,99.589744


In [71]:
px.bar(x=percent_nan.index,y=percent_nan)

### Удаление колонок с признаками

Если значения отсутствуют в достаточно большом количестве строк, то имеет смысл удалить такие колонки полностью. Например, если 99% строк имеют неопределённое значение в каком-то признаке, то этот признак не сможет использоваться для предсказывания целевой переменной, поскольку почти все данные в этом признаке неопределены. В нашем наборе данных, многие из признаков с большим количеством значений NaN по сути должны содержать значения "none" или 0. Но чтобы показать Вам различные варианты работы с отсутствующими значениями, мы удалим эти признаки вместо того, чтобы заполнить отсутствующие значения нулями или "none".

In [72]:
percent_nan.index

Index(['Lot Frontage', 'Fireplace Qu', 'Fence', 'Alley', 'Misc Feature',
       'Pool QC'],
      dtype='object')

In [73]:
df[['Lot Frontage', 'Fireplace Qu', 'Fence', 'Alley', 'Misc Feature','Pool QC']]

,Lot Frontage,Fireplace Qu,Fence,Alley,Misc Feature,Pool QC
0,141.0,Gd,NaN,NaN,NaN,NaN
1,80.0,NaN,MnPrv,NaN,NaN,NaN
2,81.0,NaN,NaN,NaN,Gar2,NaN
3,93.0,TA,NaN,NaN,NaN,NaN
4,74.0,TA,MnPrv,NaN,NaN,NaN
...,...,...,...,...,...,...
2922,37.0,NaN,GdPrv,NaN,NaN,NaN
2923,NaN,NaN,MnPrv,NaN,NaN,NaN
2924,62.0,NaN,MnPrv,NaN,Shed,NaN
2925,77.0,TA,NaN,NaN,NaN,NaN


In [74]:
df = df.drop(['Pool QC','Misc Feature','Alley','Fence'],axis=1)
df

,Order,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Lot Shape,Land Contour,Utilities,Lot Config,...,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,20,RL,141.0,31770,Pave,IR1,Lvl,AllPub,Corner,...,0,0,0,0,0,5,2010,WD,Normal,215000
1,2,20,RH,80.0,11622,Pave,Reg,Lvl,AllPub,Inside,...,0,0,120,0,0,6,2010,WD,Normal,105000
2,3,20,RL,81.0,14267,Pave,IR1,Lvl,AllPub,Corner,...,0,0,0,0,12500,6,2010,WD,Normal,172000
3,4,20,RL,93.0,11160,Pave,Reg,Lvl,AllPub,Corner,...,0,0,0,0,0,4,2010,WD,Normal,244000
4,5,60,RL,74.0,13830,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,3,2010,WD,Normal,189900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2922,2926,80,RL,37.0,7937,Pave,IR1,Lvl,AllPub,CulDSac,...,0,0,0,0,0,3,2006,WD,Normal,142500
2923,2927,20,RL,NaN,8885,Pave,IR1,Low,AllPub,Inside,...,0,0,0,0,0,6,2006,WD,Normal,131000
2924,2928,85,RL,62.0,10441,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,700,7,2006,WD,Normal,132000
2925,2929,20,RL,77.0,10010,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,4,2006,WD,Normal,170000


In [75]:
percent_nan = percent_missing(df)
percent_nan

,0
Lot Frontage,16.752137
Fireplace Qu,48.547009


In [76]:
px.bar(x=percent_nan.index,y=percent_nan)

#### Заполняем колонку Fireplace Quality на основе текстового описания

In [77]:
df['Fireplace Qu'] = df['Fireplace Qu'].fillna("None")

In [78]:
df.describe(include=['object']).T

,count,unique,top,freq
MS Zoning,2925,7,RL,2269
Street,2925,2,Pave,2913
Lot Shape,2925,4,Reg,1857
Land Contour,2925,4,Lvl,2631
Utilities,2925,3,AllPub,2922
Lot Config,2925,5,Inside,2136
Land Slope,2925,3,Gtl,2784
Neighborhood,2925,28,NAmes,443
Condition 1,2925,9,Norm,2519
Condition 2,2925,8,Norm,2896


In [79]:
percent_nan = percent_missing(df)
percent_nan

,0
Lot Frontage,16.752137


In [80]:
px.bar(x=percent_nan.index,y=percent_nan)

### Замена отсутствующих данных каким-то другим значением

Чтобы заменить отсутствующие данные в заданном признаке, нам нужно решить, какой из других признаков (без значений NaN) лучше всего коррелирует с нашим признаком. В нашем примере мы будем работать со следующими признаками:

Neighborhood: районы внутри городской черты Ames

LotFrontage: ширина фронтальной стороны дома (со стороны улицы), в футах

Мы будем работать в предположении, что признак Lot Frontage коррелирует с признаком neighborhood.

In [81]:
df['Neighborhood'].unique()

array(['NAmes', 'Gilbert', 'StoneBr', 'NWAmes', 'Somerst', 'BrDale',
       'NPkVill', 'NridgHt', 'Blmngtn', 'NoRidge', 'SawyerW', 'Sawyer',
       'Greens', 'BrkSide', 'OldTown', 'IDOTRR', 'ClearCr', 'SWISU',
       'Edwards', 'CollgCr', 'Crawfor', 'Blueste', 'Mitchel', 'Timber',
       'MeadowV', 'Veenker', 'GrnHill', 'Landmrk'], dtype=object)

In [82]:
plt.figure(figsize=(8,12))
px.box(x='Lot Frontage',y='Neighborhood',data_frame=df, range_x=[0,210], height=900) #фасад участка

<Figure size 800x1200 with 0 Axes>

### Замена отсутствующих данных на основе других признаков

Есть и более сложные методы, но обычно чем проще метод, тем лучше. Тогда мы можем не создавать модели поверх других моделей.

Про дополнительные методы замены отсутствующих значений можно почитать в этой статье: https://scikit-learn.org/stable/modules/impute.html

In [83]:
df.groupby('Neighborhood')['Lot Frontage']

In [84]:
df.groupby('Neighborhood')['Lot Frontage'].mean()

,Lot Frontage
Neighborhood,
Blmngtn,46.900000
Blueste,27.300000
BrDale,21.500000
BrkSide,55.789474
ClearCr,88.150000
CollgCr,71.336364
Crawfor,69.951807
Edwards,64.794286
Gilbert,74.207207


### Трансформация колонки

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.transform.html

In [85]:
df.head()['Lot Frontage']

,Lot Frontage
0,141.0
1,80.0
2,81.0
3,93.0
4,74.0


In [86]:
df[df['Lot Frontage'].isnull()]

,Order,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Lot Shape,Land Contour,Utilities,Lot Config,...,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
11,12,20,RL,NaN,7980,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,0,500,3,2010,WD,Normal,185000
14,15,120,RL,NaN,6820,Pave,IR1,Lvl,AllPub,Corner,...,0,0,140,0,0,6,2010,WD,Normal,212000
22,23,60,FV,NaN,7500,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,1,2010,WD,Normal,216000
23,24,20,RL,NaN,11241,Pave,IR1,Lvl,AllPub,CulDSac,...,0,0,0,0,700,3,2010,WD,Normal,149000
24,25,20,RL,NaN,12537,Pave,IR1,Lvl,AllPub,CulDSac,...,0,0,0,0,0,4,2010,WD,Normal,149900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2891,2895,20,RL,NaN,16669,Pave,IR1,Lvl,AllPub,Corner,...,0,0,0,0,0,1,2006,WD,Normal,228000
2894,2898,60,RL,NaN,11170,Pave,IR2,Lvl,AllPub,Corner,...,0,0,0,0,0,4,2006,WD,Normal,250000
2895,2899,20,RL,NaN,8098,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,10,2006,WD,Normal,202000
2909,2913,90,RL,NaN,11836,Pave,IR1,Lvl,AllPub,Corner,...,0,0,0,0,0,3,2006,WD,Normal,146500


In [87]:
df.iloc[21:26]['Lot Frontage']

,Lot Frontage
21,85.0
22,NaN
23,NaN
24,NaN
25,65.0


In [88]:
df.groupby('Neighborhood')['Lot Frontage'].transform(lambda val: val.fillna(val.mean()))

,Lot Frontage
0,141.000000
1,80.000000
2,81.000000
3,93.000000
4,74.000000
...,...
2922,37.000000
2923,75.144444
2924,62.000000
2925,77.000000


In [89]:
df.groupby('Neighborhood')['Lot Frontage'].transform(lambda val: val.fillna(val.mean())).iloc[21:26]

,Lot Frontage
21,85.000000
22,64.549383
23,75.210667
24,75.210667
25,65.000000


In [90]:
df['Lot Frontage'] = df.groupby('Neighborhood')['Lot Frontage'].transform(lambda val: val.fillna(val.mean()))

In [91]:
percent_nan = percent_missing(df)
percent_nan # было 16%

,0
Lot Frontage,0.102564


In [92]:
px.bar(x=percent_nan.index,y=percent_nan)

In [93]:
df['Lot Frontage'] = df['Lot Frontage'].fillna(0)

In [94]:
percent_nan = percent_missing(df)
percent_nan

,0


In [95]:
percent_nan

,0


Отлично! Теперь во всём нашем наборе данных нет отсутствующих значений! Имейте ввиду, что все такие трансформации лучше реализовывать в виде функций, которые потом можно легко использовать

А теперь давайте сохраним наш набор данных:

In [96]:
df.to_csv("/content/Ames_NO_Missing_Data.csv",index=False)

## Работа с категорийными (категориальными) данными (Categorical Data)


Многие методы машинного обучения не могут работать с категорийными (категориальными) данными, представленными в виде текстовых значений. Например, линейная регрессия не может вычислить коэффициент бэтта для признака со значениями цветов - "red", "blue" и т.д. Вместо этого нам нужно сконвертировать такие категории в специальные численные переменные.

## Данные

Открываем .csv-файл с очищенными данными, в которых были удалены выбросы и значения NaN.

In [97]:
df = pd.read_csv("/content/Ames_NO_Missing_Data.csv")

In [98]:
df.head()

,Order,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Lot Shape,Land Contour,Utilities,Lot Config,...,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,20,RL,141.0,31770,Pave,IR1,Lvl,AllPub,Corner,...,0,0,0,0,0,5,2010,WD,Normal,215000
1,2,20,RH,80.0,11622,Pave,Reg,Lvl,AllPub,Inside,...,0,0,120,0,0,6,2010,WD,Normal,105000
2,3,20,RL,81.0,14267,Pave,IR1,Lvl,AllPub,Corner,...,0,0,0,0,12500,6,2010,WD,Normal,172000
3,4,20,RL,93.0,11160,Pave,Reg,Lvl,AllPub,Corner,...,0,0,0,0,0,4,2010,WD,Normal,244000
4,5,60,RL,74.0,13830,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,3,2010,WD,Normal,189900


### Числовые колонки для категорий

При переводе категорийных значений в числовые значения нужно быть аккуратными и следить за тем, что числовые связи имеют смысл для модели. Например, числа в колонке MSSubClass - это просто некоторый код для каждого класса:

    MSSubClass: Identifies the type of dwelling involved in the sale.

        20	1-STORY 1946 & NEWER ALL STYLES
        30	1-STORY 1945 & OLDER
        40	1-STORY W/FINISHED ATTIC ALL AGES
        45	1-1/2 STORY - UNFINISHED ALL AGES
        50	1-1/2 STORY FINISHED ALL AGES
        60	2-STORY 1946 & NEWER
        70	2-STORY 1945 & OLDER
        75	2-1/2 STORY ALL AGES
        80	SPLIT OR MULTI-LEVEL
        85	SPLIT FOYER
        90	DUPLEX - ALL STYLES AND AGES
       120	1-STORY PUD (Planned Unit Development) - 1946 & NEWER
       150	1-1/2 STORY PUD - ALL AGES
       160	2-STORY PUD - 1946 & NEWER
       180	PUD - MULTILEVEL - INCL SPLIT LEV/FOYER
       190	2 FAMILY CONVERSION - ALL STYLES AND AGES

Здесь каждое число не связано с другими числами в этой же колонке. Например, 30 > 20, однако это не значит, что значение "1-STORY 1945 & OLDER" в каком-то смысле больше значения "1-STORY 1946 & NEWER ALL STYLES".  Хотя в других случаях вполне может быть, что например в поезде 1й класс и 2й класс вагонов действительно идут по порядку, если назначить им числа 1 и 2. Так что изучайте Ваши данные перед тем, как делать какие-то преобразования.

#### MSSubClass

In [99]:
df['MS SubClass'].unique()

array([ 20,  60, 120,  50,  85, 160,  80,  30,  90, 190,  45,  70,  75,
        40, 180, 150])

In [100]:
# Конвертация в строку
df['MS SubClass'] = df['MS SubClass'].apply(str)

### Создание "Dummy"-переменных

#### Как избежать мультиколлинеарности (наличия линейной зависимости между переменными) и ситуации "Dummy Variable Trap"

https://stats.stackexchange.com/questions/144372/dummy-variable-trap

In [101]:
person_state =  pd.Series(['Dead','Alive','Dead','Alive','Dead','Dead'])

In [102]:
person_state

,0
0,Dead
1,Alive
2,Dead
3,Alive
4,Dead
5,Dead


In [103]:
pd.get_dummies(person_state) #переход к one-hot-encoding (True - 1, False - 0)

,Alive,Dead
0,False,True
1,True,False
2,False,True
3,True,False
4,False,True
5,False,True


In [104]:
pd.get_dummies?

In [105]:
pd.get_dummies(person_state,drop_first=True)

,Dead
0,True
1,False
2,True
3,False
4,True
5,True


### Создаём Dummy-переменные на основе колонок

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.select_dtypes.html

In [106]:
df.select_dtypes(include='object') # отобразить нужный тип данных

,MS SubClass,MS Zoning,Street,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,...,Kitchen Qual,Functional,Fireplace Qu,Garage Type,Garage Finish,Garage Qual,Garage Cond,Paved Drive,Sale Type,Sale Condition
0,20,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,...,TA,Typ,Gd,Attchd,Fin,TA,TA,P,WD,Normal
1,20,RH,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,...,TA,Typ,NaN,Attchd,Unf,TA,TA,Y,WD,Normal
2,20,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,...,Gd,Typ,NaN,Attchd,Unf,TA,TA,Y,WD,Normal
3,20,RL,Pave,Reg,Lvl,AllPub,Corner,Gtl,NAmes,Norm,...,Ex,Typ,TA,Attchd,Fin,TA,TA,Y,WD,Normal
4,60,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,...,TA,Typ,TA,Attchd,Fin,TA,TA,Y,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2920,80,RL,Pave,IR1,Lvl,AllPub,CulDSac,Gtl,Mitchel,Norm,...,TA,Typ,NaN,Detchd,Unf,TA,TA,Y,WD,Normal
2921,20,RL,Pave,IR1,Low,AllPub,Inside,Mod,Mitchel,Norm,...,TA,Typ,NaN,Attchd,Unf,TA,TA,Y,WD,Normal
2922,85,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,...,TA,Typ,NaN,NaN,NaN,NaN,NaN,Y,WD,Normal
2923,20,RL,Pave,Reg,Lvl,AllPub,Inside,Mod,Mitchel,Norm,...,TA,Typ,TA,Attchd,RFn,TA,TA,Y,WD,Normal


In [107]:
df_nums = df.select_dtypes(exclude='object') # исключить столбцы текст, оставить остальные
df_objs = df.select_dtypes(include='object')

In [108]:
df_nums.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2925 entries, 0 to 2924
Data columns (total 37 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Order            2925 non-null   int64  
 1   Lot Frontage     2925 non-null   float64
 2   Lot Area         2925 non-null   int64  
 3   Overall Qual     2925 non-null   int64  
 4   Overall Cond     2925 non-null   int64  
 5   Year Built       2925 non-null   int64  
 6   Year Remod/Add   2925 non-null   int64  
 7   Mas Vnr Area     2925 non-null   float64
 8   BsmtFin SF 1     2925 non-null   float64
 9   BsmtFin SF 2     2925 non-null   float64
 10  Bsmt Unf SF      2925 non-null   float64
 11  Total Bsmt SF    2925 non-null   float64
 12  1st Flr SF       2925 non-null   int64  
 13  2nd Flr SF       2925 non-null   int64  
 14  Low Qual Fin SF  2925 non-null   int64  
 15  Gr Liv Area      2925 non-null   int64  
 16  Bsmt Full Bath   2925 non-null   float64
 17  Bsmt Half Bath

In [109]:
df_objs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2925 entries, 0 to 2924
Data columns (total 40 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   MS SubClass     2925 non-null   object
 1   MS Zoning       2925 non-null   object
 2   Street          2925 non-null   object
 3   Lot Shape       2925 non-null   object
 4   Land Contour    2925 non-null   object
 5   Utilities       2925 non-null   object
 6   Lot Config      2925 non-null   object
 7   Land Slope      2925 non-null   object
 8   Neighborhood    2925 non-null   object
 9   Condition 1     2925 non-null   object
 10  Condition 2     2925 non-null   object
 11  Bldg Type       2925 non-null   object
 12  House Style     2925 non-null   object
 13  Roof Style      2925 non-null   object
 14  Roof Matl       2925 non-null   object
 15  Exterior 1st    2925 non-null   object
 16  Exterior 2nd    2925 non-null   object
 17  Mas Vnr Type    1152 non-null   object
 18  Exter Qu

#### Конвертация

In [110]:
df_objs = pd.get_dummies(df_objs,drop_first=True)
df_objs

,MS SubClass_150,MS SubClass_160,MS SubClass_180,MS SubClass_190,MS SubClass_20,MS SubClass_30,MS SubClass_40,MS SubClass_45,MS SubClass_50,MS SubClass_60,...,Sale Type_ConLw,Sale Type_New,Sale Type_Oth,Sale Type_VWD,Sale Type_WD,Sale Condition_AdjLand,Sale Condition_Alloca,Sale Condition_Family,Sale Condition_Normal,Sale Condition_Partial
0,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,True,False,False,False,True,False
1,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,True,False,False,False,True,False
2,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,True,False,False,False,True,False
3,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,True,False,False,False,True,False
4,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,True,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2920,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,True,False
2921,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,True,False,False,False,True,False
2922,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,True,False
2923,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,True,False,False,False,True,False


In [111]:
final_df = pd.concat([df_nums,df_objs],axis=1)
final_df

,Order,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,...,Sale Type_ConLw,Sale Type_New,Sale Type_Oth,Sale Type_VWD,Sale Type_WD,Sale Condition_AdjLand,Sale Condition_Alloca,Sale Condition_Family,Sale Condition_Normal,Sale Condition_Partial
0,1,141.000000,31770,6,5,1960,1960,112.0,639.0,0.0,...,False,False,False,False,True,False,False,False,True,False
1,2,80.000000,11622,5,6,1961,1961,0.0,468.0,144.0,...,False,False,False,False,True,False,False,False,True,False
2,3,81.000000,14267,6,6,1958,1958,108.0,923.0,0.0,...,False,False,False,False,True,False,False,False,True,False
3,4,93.000000,11160,7,5,1968,1968,0.0,1065.0,0.0,...,False,False,False,False,True,False,False,False,True,False
4,5,74.000000,13830,5,5,1997,1998,0.0,791.0,0.0,...,False,False,False,False,True,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2920,2926,37.000000,7937,6,6,1984,1984,0.0,819.0,0.0,...,False,False,False,False,True,False,False,False,True,False
2921,2927,75.144444,8885,5,5,1983,1983,0.0,301.0,324.0,...,False,False,False,False,True,False,False,False,True,False
2922,2928,62.000000,10441,5,5,1992,1992,0.0,337.0,0.0,...,False,False,False,False,True,False,False,False,True,False
2923,2929,77.000000,10010,5,5,1974,1975,0.0,1071.0,123.0,...,False,False,False,False,True,False,False,False,True,False


In [112]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2925 entries, 0 to 2924
Columns: 264 entries, Order to Sale Condition_Partial
dtypes: bool(227), float64(11), int64(26)
memory usage: 1.5 MB


# Резюме

Обратите внимание, что мы не знаем, будут ли полезны все 274 колонки.

Большее количество колонок может и не привести к улучшенным результатам. Мы даже можем захотеть удалить некоторые колонки, или использовать модель с регуляризацией для выбора только важных колонок.

Сейчас же мы значительно увеличили соотношение колонок и строк, и это может снизить точность модели (однако мы не узнаем об этом до тех пор, пока не сравним разные модели и подходы).

In [113]:
final_df.corr()['SalePrice'].sort_values()

,SalePrice
Exter Qual_TA,-0.591459
Kitchen Qual_TA,-0.527461
Bsmt Qual_TA,-0.453022
Garage Finish_Unf,-0.422363
Garage Type_Detchd,-0.365209
...,...
Garage Cars,0.648488
Total Bsmt SF,0.660983
Gr Liv Area,0.727279
Overall Qual,0.802637


    OverallQual: Rates the overall material and finish of the house

           10	Very Excellent
           9	Excellent
           8	Very Good
           7	Good
           6	Above Average
           5	Average
           4	Below Average
           3	Fair
           2	Poor
           1	Very Poor

Эту колонку скорее всего заполнял риелтор, а это значит, что эта колонка может содержать обобщённую информацию с других признаков. Но если бы мы захотели использовать этот признак для предсказывания цены дома, то для каждого нового дома для применения нашей модели нам пришлось бы спрашивать риелтора!

## Сохраняем финальный набор данных

In [115]:
final_df.to_csv?

In [116]:
final_df.to_csv('/content/AMES_Final_DF.csv', index=False)

----